# POD--GPR surrogate modeling example

This notebook presents how to build and validate a POD--GPR [1] surrogate model
of microscale pollutant dispersion. This surrogate learns the dependence of the
3-D mean concentration field on meteorological forcing (the inlet wind 
direction and friction velocity) based on a dataset of precomputed LES called 
PPMLES. It can be used to significantly accelerate dispersion predictions and 
for applications that require large ensemble of model evaluations, such as data
assimilation [2].

The PPMLES dataset will soon be available online at Zenodo. 

#### Imports

In [ ]:
import h5py
import time
import numpy as np

from sklearn.decomposition import PCA
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import WhiteKernel, Matern, ConstantKernel

from auxiliaries import min_max_normalize, LogCenteredScaler, MultiOutputRegressor
from metrics import fb, nmse, fac2, mg, vg, fms

#### Inputs and data loading

In [ ]:
# Boolean to train the POD-GPR model only once
first_run = True

# POD-GPR parameters
n_modes = 10      # Number of POD modes (must be < n_train)
c_log_cut = 1e-4  # Threshold for log transformation

# GP hyperparameter
nu = 2.5          # Matern kernel order

#### Train set and test set definitions
# WARNING: PPMLES data are uniformly mixed into two feature classes (samples {0:100} and {100:200})
train_indices = np.array([k for k in range(80)] + [k for k in range(100, 180)])
test_indices = np.array([k for k in range(80, 100)] + [k for k in range(180, 200)])
n_test = np.shape(test_indices)[0]

### Load the set of time-averaged concentration fields predicted by LES
fieldsf = h5py.File('data/ave_fields.h5', 'r')
fields_train, fields_test = fieldsf['c'][train_indices], fieldsf['c'][test_indices]

# Fields preprocessing
fields_train[fields_train<0] = 0
fields_test[fields_test<0] = 0

### Load the set of input_parameters
inputf = h5py.File('data/input_parameters.h5', 'r')
alpha_train, alpha_test = inputf['alpha_inlet'][train_indices], inputf['alpha_inlet'][test_indices]
ustar_train, ustar_test = inputf['friction_velocity'][train_indices], inputf['friction_velocity'][test_indices]
n_inputs = 2

# Input parameter variation ranges
ALPHA_MIN, ALPHA_MAX = -90., 30.
USTAR_MIN, USTAR_MAX = 0.0740, 0.8875

### Load the mesh to get the dual volumes of each node
meshf = h5py.File('data/mesh.h5')
node_volumes = meshf['Nodes']['volume']

#### POD basis computation

In [ ]:
# Normalize the snapshots for the POD
fields_scaler = LogCenteredScaler(c_log_cut)
fields_scaler.fit(fields_train, node_volumes)
rescaled_fields_train = fields_scaler.transform(fields_train)

# Build the POD basis
pod = PCA(n_components=n_modes, whiten=True, random_state=0)
pod = pod.fit(rescaled_fields_train)

#### Train the Gaussian Processes

In [ ]:
# Scales the inputs
alpha_train = min_max_normalize(alpha_train, ALPHA_MIN, ALPHA_MAX)
ustar_train = min_max_normalize(ustar_train, USTAR_MIN, USTAR_MAX)
inputs_train = np.array((alpha_train, ustar_train)).T

# Compute the POD coeffs of the training samples
pod_coeffs_train = pod.transform(rescaled_fields_train)

# Define the GPs
kernel = ConstantKernel(constant_value=1.0, constant_value_bounds=[0.1, 50]) \
                        * Matern(np.ones(n_inputs), length_scale_bounds=[1e-3, 50], nu=nu) \
                        + WhiteKernel(noise_level=1e-1, noise_level_bounds=[1e-5,1])
multi_gpr = MultiOutputRegressor(GaussianProcessRegressor(kernel, 
                                                          n_restarts_optimizer=15,
                                                          random_state = 0), n_jobs=10)
multi_gpr = multi_gpr.fit(inputs_train, pod_coeffs_train)

#### POD-GPR validation on the independent test set
The POD-GPR accuracy is evaluated using the standard air quality metrics defined by [3].

In [ ]:
# Metrics array initialization
fb_list = np.zeros(n_test)
nmse_list = np.zeros(n_test)
fac2_list = np.zeros(n_test)
mg_list = np.zeros(n_test)
vg_list = np.zeros(n_test)
fms_1ppm_list = np.zeros(n_test)
fms_0_01ppm_list = np.zeros(n_test)

### POD-GPR predictions
tstart = time.time()
normalized_parameters = np.array([min_max_normalize(alpha_test, ALPHA_MIN, ALPHA_MAX), 
                                  min_max_normalize(ustar_test, USTAR_MIN, USTAR_MAX)]).T
gpr_estimates = multi_gpr.predict(normalized_parameters, return_std=False)
fields_test_pod_gpr = fields_scaler.inverse_transform(pod.inverse_transform(gpr_estimates))
integration_time = (time.time() - tstart)/n_test

### Air quality metrics computation
for i in range(n_test):
    fb_list[i] = fb(fields_test_pod_gpr[i], fields_test[i], weights=node_volumes)
    nmse_list[i] = nmse(fields_test_pod_gpr[i], fields_test[i], weights=node_volumes)
    fac2_list[i] = fac2(fields_test_pod_gpr[i], fields_test[i], c_log_cut, weights=node_volumes)
    mg_list[i] = mg(fields_test_pod_gpr[i], fields_test[i], c_log_cut, weights=node_volumes)
    vg_list[i] = vg(fields_test_pod_gpr[i], fields_test[i], c_log_cut, weights=node_volumes)
    fms_1ppm_list[i] = fms(fields_test_pod_gpr[i], fields_test[i], 1., weights=node_volumes)
    fms_0_01ppm_list[i] = fms(fields_test_pod_gpr[i], fields_test[i], 0.01, weights=node_volumes)

### Print metric scores averaged over the test set
print(f"\n=============================================================================\n\t Air quality scores averaged over all the test samples\n=============================================================================\n")
print(f"Ensemble averaged FB for the mean concentration = {np.mean(fb_list):.2f}")
print(f"Ensemble averaged NMSE for the mean concentration = {np.mean(nmse_list):.2f}")
print(f"Ensemble averaged FAC2 for the mean concentration = {np.mean(fac2_list):.2f}")
print(f"Ensemble averaged MG for the mean concentration = {np.mean(mg_list):.2f}")
print(f"Ensemble averaged VG for the mean concentration = {np.mean(vg_list):.2f}")
print(f"Ensemble averaged FMS for iso=1-ppm = {np.mean(fms_1ppm_list):.2f}")
print(f"Ensemble averaged FMS for iso=0.01-ppm = {np.mean(fms_0_01ppm_list):.2f}")
print(f"Ensemble averaged POD-GPR prediction time = {integration_time:.3f}s")

#### References

[1] Marrel, A., Perot, N., and Mottet, C. (2015). Development of a surrogate model and sensitivity analysis for spatio-temporal numerical simulators. Stochastic Environmental Research and Risk Assessment, 29(3):959–974. ISSN 1436-3259. DOI: https://doi.org/10.1007/s00477-014-0927-y.

[2] Lumet, E. (2024). Assessing and reducing uncertainty in large-eddy simulation for microscale atmospheric dispersion. PhD thesis, Université Toulouse III - Paul Sabatier. URL: https://theses.fr/2024TLSES003. Accessed: 2024-07-08.

[3] Chang, J. and Hanna, S. (2004). Air quality model performance evaluation. Meteorol. Atm. Phys, 87(1):167–196. DOI: https://doi.org/10.1007/s00703-003-0070-7.